In [1]:
import pandas as pd
import numpy as np
from skfuzzy import control as ctrl
from skfuzzy import membership as mf

In [2]:
#example meals database

meals = pd.DataFrame(data = {
        'calories':[300, 650, 800, 600, 400, 550],
        'protein':[10, 25, 20, 15, 18, 30],
        'fiber':[12, 4, 2, 1, 5, 6], 
        'price':[8, 7, 12, 6, 5, 10]
        }, index = ['salad', 'chicken rice', 'burger', 'fried nuggets', 'sandwich', 'grilled fish'])

In [3]:
#input variables
caloryDiff = ctrl.Antecedent(np.arange(0, 1000, 1), 'caloryDiff')
proteinDiff = ctrl.Antecedent(np.arange(0, 50, 1), 'proteinDiff')
fiberDiff = ctrl.Antecedent(np.arange(0, 20, 1), 'fiberDiff')
# priceDiff = ctrl.Antecedent(np.arange(0, 20, 1), 'priceDiff')

#output variable
suitability = ctrl.Consequent(np.arange(0, 10, 1), 'suitability')

In [4]:
#membership functions
caloryDiff['low'] = mf.trimf(caloryDiff.universe, [0, 0, 400])
caloryDiff['mid'] = mf.trimf(caloryDiff.universe, [300, 500, 700])
caloryDiff['high'] = mf.trapmf(caloryDiff.universe, [600, 800, 1000, 1000])

proteinDiff['low'] = mf.trimf(proteinDiff.universe, [0, 0, 15])
proteinDiff['mid'] = mf.trimf(proteinDiff.universe, [10, 25, 35])
proteinDiff['high'] = mf.trapmf(proteinDiff.universe, [30, 40, 50, 50])

fiberDiff['low'] = mf.trimf(fiberDiff.universe, [0, 0, 5])
fiberDiff['mid'] = mf.trimf(fiberDiff.universe, [4, 8, 12])
fiberDiff['high'] = mf.trapmf(fiberDiff.universe, [10, 15, 20, 20])

# priceDiff['low'] = mf.trapmf(priceDiff.universe, [0, 0, 5, 7])
# priceDiff['mid'] = mf.trimf(priceDiff.universe, [5, 10, 13])
# priceDiff['high'] = mf.trapmf(priceDiff.universe, [10, 15, 20, 20])

suitability['not'] = mf.trimf(suitability.universe, [0, 0, 2])
suitability['slightly'] = mf.trimf(suitability.universe, [1, 3, 4])
suitability['kinda'] = mf.trimf(suitability.universe, [3, 5, 6])
suitability['very'] = mf.trimf(suitability.universe, [5, 7, 8])
suitability['match'] = mf.trapmf(suitability.universe, [7, 9, 10, 10])

In [5]:
#system rules

rules = [
    ctrl.Rule(caloryDiff['low'] & proteinDiff['low'] & fiberDiff['low'], (suitability['match'])),
    ctrl.Rule(caloryDiff['low'] & proteinDiff['low'] & fiberDiff['mid'], (suitability['very'])),
    ctrl.Rule(caloryDiff['low'] & proteinDiff['low'] & fiberDiff['high'], (suitability['very'])),

    ctrl.Rule(caloryDiff['low'] & proteinDiff['mid'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(caloryDiff['low'] & proteinDiff['mid'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(caloryDiff['low'] & proteinDiff['mid'] & fiberDiff['high'], (suitability['kinda'])),

    ctrl.Rule(caloryDiff['low'] & proteinDiff['high'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(caloryDiff['low'] & proteinDiff['high'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(caloryDiff['low'] & proteinDiff['high'] & fiberDiff['high'], (suitability['slightly'])),

    ctrl.Rule(caloryDiff['mid'] & proteinDiff['low'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(caloryDiff['mid'] & proteinDiff['low'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(caloryDiff['mid'] & proteinDiff['low'] & fiberDiff['high'], (suitability['kinda'])),

    ctrl.Rule(caloryDiff['mid'] & proteinDiff['mid'] & fiberDiff['low'], (suitability['kinda'])),
    ctrl.Rule(caloryDiff['mid'] & proteinDiff['mid'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(caloryDiff['mid'] & proteinDiff['mid'] & fiberDiff['high'], (suitability['kinda'])),

    ctrl.Rule(caloryDiff['mid'] & proteinDiff['high'] & fiberDiff['low'], (suitability['kinda'])),
    ctrl.Rule(caloryDiff['mid'] & proteinDiff['high'] & fiberDiff['mid'], (suitability['slightly'])),
    ctrl.Rule(caloryDiff['mid'] & proteinDiff['high'] & fiberDiff['high'], (suitability['not'])),

    ctrl.Rule(caloryDiff['high'] & proteinDiff['low'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(caloryDiff['high'] & proteinDiff['low'] & fiberDiff['mid'], (suitability['slightly'])),
    ctrl.Rule(caloryDiff['high'] & proteinDiff['low'] & fiberDiff['high'], (suitability['slightly'])),

    ctrl.Rule(caloryDiff['high'] & proteinDiff['mid'] & fiberDiff['low'], (suitability['slightly'])),
    ctrl.Rule(caloryDiff['high'] & proteinDiff['mid'] & fiberDiff['mid'], (suitability['not'])),
    ctrl.Rule(caloryDiff['high'] & proteinDiff['mid'] & fiberDiff['high'], (suitability['not'])),

    ctrl.Rule(caloryDiff['high'] & proteinDiff['high'] & fiberDiff['low'], (suitability['slightly'])),
    ctrl.Rule(caloryDiff['high'] & proteinDiff['high'] & fiberDiff['mid'], (suitability['not'])),
    ctrl.Rule(caloryDiff['high'] & proteinDiff['high'] & fiberDiff['high'], (suitability['not'])),
]

food_ctrl = ctrl.ControlSystem(rules=rules)

food = ctrl.ControlSystemSimulation(control_system=food_ctrl)

In [6]:
#user input how many nutrient they want
calories = 0
protein = 0
fiber = 0
budget = 10

result = []
result.clear()

#calculate the difference in nutrients to get suitability
for meal in meals.itertuples():
    food.input['caloryDiff'] = abs(calories - meal.calories)
    food.input['proteinDiff'] = abs(protein - meal.protein)
    food.input['fiberDiff'] = abs(fiber - meal.fiber)

    food.compute()
    result.append([meal.Index, food.output['suitability']])


print(pd.DataFrame(result))

               0         1
0          salad  6.559524
1   chicken rice  3.548936
2         burger  2.628571
3  fried nuggets  4.577778
4       sandwich  4.559524
5   grilled fish  4.611111
